In [1]:
import numpy as np
import scipy as sp
from scipy import io,integrate,sparse,signal

import matplotlib.pyplot as plt

import os,sys
sys.path.insert(0, '..')

from partial_trace import *
#from lanczos_bin import mystep,distribution

from IPython.display import clear_output
np.set_printoptions(linewidth=300)
%load_ext autoreload
%autoreload 2

In [2]:
s = 1/2

M = int(2*s+1)

Nc = 3 # number of clusters

N = Nc*5
N_s = 5
N_b = N-N_s
d_a = M**N_s
d_b = M**N_b

In [3]:
# Generate the adjacency matrix for Kagome strip chain
Jcx = np.zeros((5,5))
Jcx[0,1:] = 1
Jcx[1:,0] = 1

Jx = np.kron(np.eye(Nc),Jcx)

Jc1 = np.zeros((5,5))
Jc1[1,2] = 1
Jc1[3,4] = 1
Jc1 += Jc1.T

J1 = np.kron(np.eye(Nc),Jc1)

J2 = np.zeros((N,N))
for i in range(Nc):
    J2[i*5+2,((i+1)*5+1)%N] = 1
    J2[i*5+4,((i+1)*5+3)%N] = 1

J2 += J2.T

In [5]:
J1_strength = 1
J2_strength = .1 # Change the strength to vary the couplings

J = Jx + J1_strength*J1 + J2_strength*J2

In [6]:
# Builde Hamiltonian
H1 = get_hamiltonian(J,J,J,0,s)
H2 = get_hamiltonian(0*J,0*J,0*J,1,s)

In [7]:
# Find the jumping points at the ground states
h_min = 0
h_max = 3
iterations = 10
tolerance = 2e-2
turning_points = []
binary_search_recursive(lambda h: ground_state_function_smooth(h,H1,H2,d_a,d_b,k=5,β=1e5), h_min, h_max, iterations, tolerance, turning_points)
turning_points = np.hstack([[h_min],turning_points,[h_max]])

0,3,1.819e+00,2.375e-03,7.801e-01,10
0,1.5,1.819e+00,7.801e-01,7.801e-01,9
0,0.75,1.819e+00,7.801e-01,7.801e-01,8
0,0.375,1.819e+00,7.801e-01,1.732e+00,7
0,0.1875,1.819e+00,1.732e+00,1.732e+00,6
0,0.09375,1.819e+00,1.732e+00,1.732e+00,5
0,0.046875,1.819e+00,1.732e+00,1.732e+00,4
0,0.0234375,1.819e+00,1.732e+00,1.732e+00,3
0,0.01171875,1.819e+00,1.732e+00,1.732e+00,2
0,0.005859375,1.819e+00,1.732e+00,1.732e+00,1
0.1875,0.375,1.732e+00,7.801e-01,1.732e+00,6
0.28125,0.375,1.732e+00,7.801e-01,7.801e-01,5
0.28125,0.328125,1.732e+00,7.801e-01,1.732e+00,4
0.3046875,0.328125,1.732e+00,7.801e-01,1.732e+00,3
0.31640625,0.328125,1.732e+00,7.801e-01,7.801e-01,2
0.31640625,0.322265625,1.732e+00,7.801e-01,1.732e+00,1
1.5,3,7.801e-01,2.375e-03,6.419e-01,9
1.5,2.25,7.801e-01,6.419e-01,7.801e-01,8
1.875,2.25,7.801e-01,6.419e-01,1.130e+00,7
1.875,2.0625,7.801e-01,1.130e+00,7.801e-01,6
1.96875,2.0625,7.801e-01,1.130e+00,1.130e+00,5
1.96875,2.015625,7.801e-01,1.130e+00,7.801e-01,4
1.9921875,2.015625,7.801

In [11]:
turning_points

array([0.00000000e+00, 1.46484375e-03, 3.20800781e-01, 1.99365234e+00, 2.24853516e+00, 2.37451172e+00, 3.00000000e+00])

In [8]:
acc = (h_max-h_min)/2**iterations # accuracy of turning points
total_nodes = 100
num_nodes,hs = scaled_cheby(turning_points, total_nodes) # use chebyshev nodes to interpolate points between
hs = np.hstack([turning_points[1:]-acc,turning_points[:-1]+acc,hs]) # add points to the left and right of the boundaries

In [9]:
# save system parameters
directory = f'data'
np.save(f'{directory}/dimensions.npy',[s,N,N_s],allow_pickle=True)
np.save(f'{directory}/couplings.npy',[J,J,J],allow_pickle=True)
sp.sparse.save_npz(f'{directory}/H1',H1)
sp.sparse.save_npz(f'{directory}/H2',H2)
np.save(f'{directory}/turning_points.npy',turning_points)
np.save(f'{directory}/num_nodes.npy',num_nodes)

In [10]:
# Set the number of eigenvalues and deflations
k = 5
m = 25

In [13]:
# run the driver file to generate all the data files needed for the plot.
data_directory = f'data'
os.makedirs(f'{directory}/{data_directory}',exist_ok=True) # make subdirectory for data files

out_all = []
for hi,h in enumerate(hs):
    experiment_name = f'{h}'
    print(f'field strength: {hi} of {len(hs)}, {experiment_name}')
    clear_output(wait=True)
    os.system(f'python vN_entropy_phase_driver.py {h} {k} {m} {directory} {experiment_name} {data_directory}')

/Users/muddy_flesh/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
